In [15]:
import json
import os
from io import StringIO

with open('./models/20200310_172258/model/timeline.json') as f:
    #io = StringIO(f)
    f1 = json.load(f)
    print(f1['traceEvents'][0])
    traceEvents = f1['traceEvents'][0]
    traceEvents2 = f1['traceEvents'][1]
    
    newTraceEvents = {'traceEvents': [traceEvents, traceEvents2]}
    print(json.dumps(newTraceEvents))

{'name': 'process_name', 'ph': 'M', 'pid': 0, 'args': {'name': 'Allocators'}}
{"traceEvents": [{"name": "process_name", "ph": "M", "pid": 0, "args": {"name": "Allocators"}}, {"name": "process_name", "ph": "M", "pid": 1, "args": {"name": "/host:CPU Compute"}}]}


In [9]:
!head ./models/20200310_172258/model/timeline.json -n 20

{
    "traceEvents": [
        {
            "name": "process_name",
            "ph": "M",
            "pid": 0,
            "args": {
                "name": "Allocators"
            }
        },
        {
            "name": "process_name",
            "ph": "M",
            "pid": 1,
            "args": {
                "name": "/host:CPU Compute"
            }
        },
        {
            "name": "process_name",


In [4]:
import tensorflow as tf
tf.version.VERSION.startswith('2')

True

In [26]:
from google.cloud import bigquery

PROJECT_ID = "alekseyv-scalableai-dev"
dataset_id = 'criteo_kaggle'
table_id = 'tf114_profile'

client = bigquery.Client(project=PROJECT_ID)

dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
job_config.autodetect = True
filename = '/usr/local/google/home/alekseyv/vlasenkoalexey/columnar_estimator_sample/models/20200312_095140/model/timeline.json'

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)
    
job.result()  # Waits for table load to complete.

print("Loaded {} rows into {}:{}.".format(job.output_rows, dataset_id, table_id))    

/usr/local/lib/python3.7/dist-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


BadRequest: 400 Provided Schema does not match Table alekseyv-scalableai-dev:criteo_kaggle.tf114_profile. Cannot add fields (field: traceEvents)

In [37]:
from google.api_core.exceptions import GoogleAPIError

def create_bigquery_dataset_if_necessary(dataset_id):
    # Construct a full Dataset object to send to the API.
    client = bigquery.Client(project=PROJECT_ID)
    dataset = bigquery.Dataset(bigquery.dataset.DatasetReference(PROJECT_ID, dataset_id))
    dataset.location = 'US'

    try:
        dataset = client.create_dataset(dataset)  # API request
        return True
    except GoogleAPIError as err:
        if err.code != 409: # http_client.CONFLICT
              raise
    return False

def import_json_file(dataset_id, table_id, file_name):
    client = bigquery.Client(project=PROJECT_ID)

    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
    job_config.autodetect = True

    with open(file_name, "rb") as source_file:
        job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

    job.result()  # Waits for table load to complete.

    print("Loaded {} rows into {}:{}.".format(job.output_rows, dataset_id, table_id))    

In [40]:
#create_bigquery_dataset_if_necessary('traces')
import_json_file('traces', 'tf115', '/usr/local/google/home/alekseyv/vlasenkoalexey/columnar_estimator_sample/models/20200312_154614/model/timeline.json')

/usr/local/lib/python3.7/dist-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Loaded 1 rows into traces:tf115.


In [12]:
%load_ext google.cloud.bigquery

In [41]:
%%bigquery --project alekseyv-scalableai-dev --verbose --destination_table traces.tf115_processed

SELECT 
index, 
traceEvent.ph, 
traceEvent.cat, 
traceEvent.name, 
traceEvent.pid,
traceEvent.tid,
traceEvent.ts,
traceEvent.dur,
traceEvent.args.name as arg_name,
traceEvent.args.op as arg_op 
FROM 
`alekseyv-scalableai-dev.traces.tf115` AS t, 
UNNEST(t.traceEvents) as traceEvent WITH OFFSET AS index;


Executing query with job ID: c1c7695a-63c1-4541-81fe-f570aedc4e33
Query executing: 0.48s
Query complete after 2.45s


,index,ph,cat,name,pid,tid,ts,dur,arg_name,arg_op
0,38,X,Op,unknown,1,4.0,1.584053e+15,3.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
1,96,X,Op,unknown,1,4.0,1.584053e+15,3.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
2,143,X,Op,unknown,1,5.0,1.584053e+15,4.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
3,163,X,Op,unknown,1,5.0,1.584053e+15,4.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
4,179,X,Op,unknown,1,5.0,1.584053e+15,2.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
...,...,...,...,...,...,...,...,...,...,...
49718,49627,X,Op,unknown,1,5.0,1.584053e+15,1.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
49719,49640,X,Op,unknown,1,4.0,1.584053e+15,6.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
49720,49643,X,Op,unknown,1,6.0,1.584053e+15,15.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
49721,49653,X,Op,unknown,1,3.0,1.584053e+15,6.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown


In [49]:
%%bigquery --project alekseyv-scalableai-dev tf115_processed

SELECT 
sum(dur) as total_dur, 
avg(dur) as avg_dur,
min(dur) as min_dur,
max(dur) as max_dur, 
arg_name 
FROM `alekseyv-scalableai-dev.traces.tf115_processed` 
group by arg_name order by 1 desc
limit 20;


In [57]:
from IPython.display import display, HTML

display(HTML(tf115_processed.to_html()))

,total_dur,avg_dur,min_dur,max_dur,arg_name
0,9261198,4.630599e+05,7,9260747,OneShotIterator
1,9227555,9.227555e+06,9227555,9227555,OptimizeDataset/list_files/MatchingFiles:MatchingFiles
2,805513,4.027565e+04,197,541780,IteratorGetNext
3,801583,4.007915e+04,26,541570,Iterator::Model
4,801476,4.007380e+04,25,541502,Iterator::Model::Map
5,800987,4.004935e+04,2,541472,Iterator::Model::Map::Prefetch
6,622289,6.220402e+01,0,218085,Iterator::Model::Map::Prefetch::ParseExample::BatchV2::Shuffle::ParallelInterleave[0]::FlatMap
7,619304,6.190564e+01,0,218083,Iterator::Model::Map::Prefetch::ParseExample::BatchV2::Shuffle::ParallelInterleave[0]::FlatMap[0]::TFRecord
8,604796,4.031973e+04,1,541271,Iterator::Model::Map::Prefetch::ParseExample
9,594405,5.403682e+04,4340,539151,Iterator::Model::Map::Prefetch::ParseExample::BatchV2


In [58]:
import_json_file('traces', 'tf114', '/usr/local/google/home/alekseyv/vlasenkoalexey/columnar_estimator_sample/models/20200312_155239/model/timeline.json')

/usr/local/lib/python3.7/dist-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Loaded 1 rows into traces:tf114.


In [59]:
%%bigquery --project alekseyv-scalableai-dev --verbose --destination_table traces.tf114_processed

SELECT 
index, 
traceEvent.ph, 
traceEvent.cat, 
traceEvent.name, 
traceEvent.pid,
traceEvent.tid,
traceEvent.ts,
traceEvent.dur,
traceEvent.args.name as arg_name,
traceEvent.args.op as arg_op 
FROM 
`alekseyv-scalableai-dev.traces.tf114` AS t, 
UNNEST(t.traceEvents) as traceEvent WITH OFFSET AS index;

Executing query with job ID: 367c52cf-b38e-41c8-b0b7-92aaf84f554f
Query executing: 1.73s
Query complete after 2.07s


,index,ph,cat,name,pid,tid,ts,dur,arg_name,arg_op
0,46,X,Op,unknown,1,4.0,1.584054e+15,1.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
1,57,X,Op,unknown,1,1.0,1.584054e+15,1.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
2,154,X,Op,unknown,1,4.0,1.584054e+15,1.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
3,161,X,Op,unknown,1,1.0,1.584054e+15,1.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
4,185,X,Op,unknown,1,1.0,1.584054e+15,1.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
...,...,...,...,...,...,...,...,...,...,...
51470,51160,X,Op,unknown,1,7.0,1.584054e+15,13.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
51471,51197,X,Op,unknown,1,8.0,1.584054e+15,0.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
51472,51314,X,Op,unknown,1,7.0,1.584054e+15,3.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown
51473,51334,X,Op,unknown,1,7.0,1.584054e+15,0.0,Iterator::Model::Map::Prefetch::ParseExample::...,unknown


In [60]:
%%bigquery --project alekseyv-scalableai-dev tf114_processed

SELECT 
sum(dur) as total_dur, 
avg(dur) as avg_dur,
min(dur) as min_dur,
max(dur) as max_dur, 
arg_name 
FROM `alekseyv-scalableai-dev.traces.tf114_processed` 
group by arg_name order by 1 desc
limit 20;

In [61]:
from IPython.display import display, HTML

display(HTML(tf114_processed.to_html()))

,total_dur,avg_dur,min_dur,max_dur,arg_name
0,504993,3079.225610,0,504125,Iterator::Model::Map::Prefetch::ParseExample::Batch::Shuffle::ParallelInterleave[2]::FlatMap
1,504825,3078.201220,0,504007,Iterator::Model::Map::Prefetch::ParseExample::Batch::Shuffle::ParallelInterleave[2]::FlatMap[0]::TFRecord
2,499071,29357.117647,2,409373,Iterator::Model::Map::Prefetch::ParseExample
3,465509,23275.450000,180,410073,IteratorGetNext
4,462494,23124.700000,22,409925,Iterator::Model
5,462302,23115.100000,19,409787,Iterator::Model::Map
6,461882,23094.100000,1,409757,Iterator::Model::Map::Prefetch
7,461456,41950.545455,3751,407325,Iterator::Model::Map::Prefetch::ParseExample::Batch
8,451106,85.859536,0,403024,Iterator::Model::Map::Prefetch::ParseExample::Batch::Shuffle
9,440709,28.891373,0,328462,Iterator::Model::Map::Prefetch::ParseExample::Batch::Shuffle::ParallelInterleave
